## Preprocès: ##

- Eliminar els digits del text
- Convertir tot el text a minúscula
- Substitueix els espais en blanc continus per un de sol
- Concatena totes les frases amb un espai doble al mig 


In [ ]:
import re 
from nltk.collocations import TrigramCollocationFinder
from collections import Counter, defaultdict
import os
import random

In [ ]:
def preprocess_text(text):
    text = re.sub(r'\d+', '', text)  
    text = text.lower() 
    text = re.sub(r'\s+', ' ', text).strip()  
    sentences = text.split('.') 
    return '  '.join(sentences)

def load_and_preprocess_data(data_path):
    with open(data_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return preprocess_text(text)

languages = ['deu', 'eng', 'fra', 'ita', 'nld', 'spa']
path = 'C:/Users/Paess/Documents/Uni/Segon/PLH'
train = {}
test = {}

for idiom in languages:
    file_path = os.path.join(path, f'{idiom}_trn.txt')
    text = load_and_preprocess_data(file_path)
    train[idiom] = text

for idiom in languages:
    file_path = os.path.join(path, f'{idiom}_tst.txt')
    text = load_and_preprocess_data(file_path)
    test[idiom] = text

train_split = {}
val_split = {}

for idiom in languages:
    text = train[idiom]
    split_index = int(len(text) * 0.8)
    train_split[idiom] = text[:split_index]
    val_split[idiom] = text[split_index:]


In [16]:
def generate_trigrams(text_list):
    all_trigrams = []
    for text in text_list:
        words = text.split()  
        finder = TrigramCollocationFinder.from_words(words)
        all_trigrams.extend([''.join(trigram) for trigram in finder.ngram_fd.keys()])
    return all_trigrams

def frequent_trigrams(trigrams):
    trigram_counts = Counter(trigrams)
    frequent_trigrams_set = {trigram for trigram, count in trigram_counts.items() if count >= 5}
    return [trigram for trigram in trigrams if trigram in frequent_trigrams_set]

for idiom in train_split: 
    train_split[idiom] = frequent_trigrams(generate_trigrams(train_split[idiom]))
    val_split[idiom] = frequent_trigrams(generate_trigrams(val_split[idiom]))

